In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import pageviewapi
import datetime as dt
from bs4 import BeautifulSoup
import altair as alt

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [5]:
import numpy as np

---

### data from the national council of state legislatures

In [6]:
# url = "https://www.ncsl.org/research/civil-and-criminal-justice/clearing-criminal-records-for-cannabis-offenses.aspx"

In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup


html = urlopen(
    "https://www.ncsl.org/research/civil-and-criminal-justice/clearing-criminal-records-for-cannabis-offenses.aspx"
)

In [8]:
soup = BeautifulSoup(html, "html.parser")

In [9]:
states = soup.find_all("div", class_="notshowing")

In [10]:
names = []
record_clearing = []
cannabis_specific = []
citation = []
proposed_law = []
recreational_use = []
medical_use = []

for state in states[0:51]:
    names.append(state.find("h4").text)
    data = state.find_all("p")
    record_clearing.append(data[0].text)
    if "Specific" in data[1].text:
        cannabis_specific.append(data[1].text)
        citation.append(data[2].text)
    else:
        cannabis_specific.append("NaN")
        citation.append(data[1].text)
    if "Proposed" in data[2].text:
        proposed_law.append(data[2].text)
    else:
        if "Proposed" in data[3].text:
            proposed_law.append(data[3].text)
        else:
            proposed_law.append("NaN")
    if "Recreational" in data[2].text:
        recreational_use.append(data[2].text)
    else:
        if "Recreational" in data[3].text:
            recreational_use.append(data[3].text)
        else:
            if len(data) > 4:
                if "Recreational" in data[4].text:
                    recreational_use.append(data[4].text)
    if "Medical" in data[3].text:
        medical_use.append(data[3].text)
    else:
        if len(data) > 4:
            if "Medical" in data[4].text:
                medical_use.append(data[4].text)
            else:
                if len(data) > 5:
                    if "Medical" in data[5].text:
                        medical_use.append(data[5].text)


#    for item in data:
#        if "Specific" in data[1].text:
#            cannabis_specific.append(data[1].text)
#            citation.append(data[2].text)
#        else:
#            cannabis_specific.append("NaN")
#            citation.append(data[1])

In [11]:
len(medical_use)

51

In [12]:
df = pd.DataFrame(
    {
        "state": names,
        "record_clearing": record_clearing,
        "cannabis_specific": cannabis_specific,
        "citation": citation,
        "proposed_law": proposed_law,
        "recreational_use": recreational_use,
        "medical_use": medical_use,
    }
)

In [13]:
df["record_clearing"] = df["record_clearing"].str.replace(
    "Cannabis Record Clearing: ", ""
)

df["citation"] = df["citation"].str.replace("Citation: ", "")

df["cannabis_specific"] = df["cannabis_specific"].str.replace("Cannabis Specific: ", "")

df["proposed_law"] = df["proposed_law"].str.replace("Proposed Legislation: ", "")

df["proposed_law"] = df["proposed_law"].str.replace("Proposed Ballot Initiative: ", "")

df["recreational_use"] = df["recreational_use"].str.replace("Recreational Use: ", "")

df["medical_use"] = df["medical_use"].str.replace("Medical Use: ", "")

In [15]:
df.value_counts("medical_use")

medical_use
Yes               32
No                13
Decriminalized     4
Y                  2
dtype: int64

In [16]:
df.value_counts("recreational_use")

recreational_use
No                21
Decriminalized    16
Yes               11
N                  1
Y                  1
Yes; No Sales      1
dtype: int64

### get the more recent tables

In [ ]:
html = urlopen(
    "https://www.ncsl.org/research/health/state-medical-marijuana-laws.aspx"
)